In [1]:
import cv2
import pickle
import numpy as np

# Function to load parking positions from a file
def load_parking_positions(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

# Function to check parking spaces and draw on the image
def check_parking_space(img, imgPro, posList, width, height):
    spaceCounter = 0

    for pos in posList:
        x, y = pos

        # Extract the region of interest (ROI) for each parking space
        imgCrop = imgPro[y:y + height, x:x + width]

        # Use image processing techniques to analyze the ROI
        count = cv2.countNonZero(imgCrop)

        # Customize the display based on occupancy
        if count < 900:
            color = (0, 255, 0)  # Green color for available spaces
            thickness = 5
            spaceCounter += 1
        else:
            color = (0, 0, 255)  # Red color for occupied spaces
            thickness = 2

        # Draw rectangles around parking spaces and display occupancy count
        cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), color, thickness)
        cv2.putText(img, str(count), (x, y + height - 3), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Display the total number of free parking spaces
    cv2.putText(img, f'Free: {spaceCounter}/{len(posList)}', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 200, 0), 5)

# Main function to process the video feed
def main(video_filename, parking_positions_filename, width, height):
    def empty(a):
        pass

    cap = cv2.VideoCapture(video_filename)

    if not cap.isOpened():
        print("Error: Could not open the video file.")
        return

    posList = load_parking_positions(parking_positions_filename)

    cv2.namedWindow("Vals")
    cv2.createTrackbar("Val1", "Vals", 25, 50, empty)
    cv2.createTrackbar("Val2", "Vals", 16, 50, empty)
    cv2.createTrackbar("Val3", "Vals", 5, 50, empty)

    while True:
        ret, img = cap.read()

        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue

        # Preprocess the frame for better parking space detection
        imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        imgBlur = cv2.GaussianBlur(imgGray, (3, 3), 1)

        val1 = cv2.getTrackbarPos("Val1", "Vals")
        val2 = cv2.getTrackbarPos("Val2", "Vals")
        val3 = cv2.getTrackbarPos("Val3", "Vals")

        if val1 % 2 == 0:
            val1 += 1
        if val3 % 2 == 0:
            val3 += 1

        imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, val1, val2)
        imgMedian = cv2.medianBlur(imgThreshold, val3)
        kernel = np.ones((3, 3), np.uint8)
        imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)

        # Use the parking space checking function
        check_parking_space(img, imgDilate, posList, width, height)

        # Display the frame with parking space information
        cv2.imshow("Parking Spaces", img)

        if cv2.waitKey(10) == 27:  # Press 'Esc' key to exit
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_filename = 'carPark.mp4'
    parking_positions_filename = 'CarParkPos'
    width = 107
    height = 48

    main(video_filename, parking_positions_filename, width, height)
